# Art-Deco DSLs: beyond operator overloading

I think one of the greatest ideas in computer science is the notion of an **"embedded DSL"**: the idea that programming languages come in all shapes and sizes, and that little domain-specific languages (DSLs) can fit nicely within larger general-purpose languages. For example, many popular array-programming DSLs like [NumPy](https://numpy.org/), [PyTorch](https://pytorch.org/) and [JAX](https://github.com/jax-ml/jax) live _within_ the general-purpose language Python.

Python in particular is a great language to build embedded DSLs in, because DSLs can use Python's elegant [_operator overloading interface_](https://docs.python.org/3/reference/datamodel.html#special-method-names) to repurpose existing Python syntax for their own specialized semantics (e.g. repurposing the `+` operator for vectorized array operations). There is a lot to like about embedding DSLs this way:
1. Embedded DSLs inherit a lot of comforts "for free" from having Python as the host language (e.g. parsing, error checking, editor tooling, syntax highlighting, built-in data types, package registry, …).
2. Embedded DSLs immediately integrate with the Python language, and even the broader module ecosystem (including other DSLs). For example, you can do a computation with NumPy or PyTorch and then plot the result with Matplotlib.
3. Potential users often already know Python, so it is not hard to convince them to try an embedded DSL.

So why _wouldn't_ you implement your next DSL by operator-overloading in Python?

**One fundamental limitation of operator overloading** is that it cannot help you override Python's semantics for **control flow** (e.g. the semantics of the `if` statement) or **scoping** (e.g. the semantics of variable assignment). This often leads to awkward, unergonomic designs. For example, because JAX cannot override Python's `if` or `while` statements by operator overloading, conditionals and loops are [messy](https://jax.readthedocs.io/en/latest/control-flow.html#control-flow) to work with. Similarly, a probabilistic programming language really does need traverse both branches of an `if` statement whose predicate depends on a random choice—but you cannot implement that with pure operator overloading.

For this reason, many new DSLs over the last few years (e.g. [SEJITS](https://sejits.eecs.berkeley.edu/), [Taichi](https://github.com/taichi-dev/taichi), [Numba](https://numba.pydata.org/), [Exo](https://github.com/exo-lang/exo), [memo](https://github.com/kach/memo), [Dr.Jit](https://github.com/mitsuba-renderer/drjit), [Allo](https://github.com/cornell-zhang/allo), [loma](https://github.com/BachiLi/loma_public)) have taken **a clever new approach to embedding**: _parsing and transforming Python ASTs directly within Python_, using Python's built-in mechanisms for introspection. These DSLs work by providing a Python [decorator](https://docs.python.org/3/glossary.html#term-decorator) that is used to mark a definition as DSL code rather than Python code. The decorators never intend to actually execute the definition they decorate. Instead, they do something sneaky: they read in the decorated definition's Python AST, and then compile that AST it _as if_ it were code in the DSL, i.e. under the DSL's artificial semantics rather than Python's natural semantics. This creates the illusion of the DSL's code running seamlessly within Python. I think of these as "artificial decorators"; hence, among friends I call them **"Art-Deco DSLs."**

This pattern is _very_ powerful. It lets us build arbitrarily complex DSLs on top of Python's rich syntax, borrowing as much or as little of Python's existing semantics as we wish. For example, I will show you how to build a simple toy DSL called "env" which augments Python such that all-caps variable names get automatically looked up in the shell's environment (as in shell scripts).

```python
@env
def whoami():
    return "You are: " + USER
print(whoami())
```

The env DSL is very simple and just extends Python's scoping behavior while preserving the rest of the semantics as-is. Other DSLs might rewrite the semantics entirely. For example, we could build an Art-Deco implementation of Conway's [FRACTRAN language](https://en.wikipedia.org/wiki/FRACTRAN#). Here, a sequence of Python integer literals and division operators gets interpreted _as if_ it were FRACTRAN code.

```python
@fractran
def multiply(a, b):
    455/33
    11/13
    1/11
    3/7
    11/2
    1/3
print(multiply(3, 4))  # 12
```

Similarly, we might creatively reinterpret Python's syntax to provide a convenient Graphviz-like syntax for defining graphs. This language doesn't really have _any_ runtime semantics to speak of: it's just syntactic sugar to describe data in a convenient format.

```python
@digraph
def G():
    a >> b >> c
    a >> c (linestyle=dashed)
```

I think the Art-Deco pattern is really worth knowing, and so for future DSL designers I've compiled some notes on how to build DSLs this way. Most of this is straightforward, and I would expect that from this description alone a proficient Python programmer could come up with the rest of this guide on their own by consulting the Python documentation. But there are some tricky bits here and there to watch out for, and I think there is value in collecting everything all in one page…

## Decorators

A [decorator](https://docs.python.org/3/glossary.html#term-decorator) is a Python "wrapper" function that is applied to another function as soon as it is defined. To apply decorator `deco` to function `foo`'s definition, you prefix `foo`'s definition with `@deco`. This calls decorator `deco` with function `foo` as the argument, i.e. it runs `deco(foo)`.

In [1]:
def deco(f):  # Definition of decorator (just a function)
    print(f'Called decorator on function "{f.__name__}".')

@deco  # Decorator applied to definition of "foo"
def foo():  # Definition of "foo"
    print("Hello, world!")

Called decorator on function "foo".


Notice that the function `foo` never actually ran (that is, the text "Hello, world!" was never printed).

The return value of a decorator replaces definition it decorates. Hence, if a decorator itself returns a function, it creates the illusion of replacing that function with a modified version of itself. For example, here is a decorator `@diff` that replaces the given function `f` with an approximation of its derivative `df`. To do this, the decorator defines a new function `df`, which internally calls the original `f` at `x` and `x + dx` to compute a finite-difference estimate of the derivative. Then the decorator returns `df`, which means `f` is replaced with `df`. Notice that `df` is a totally different function from `f` — it even has a different signature! They just happen to be related in the sense that `df` calls `f` twice to compute the finite difference. In general, a decorator can return an arbitrary function (or even an object that isn't a function at all).

In [2]:
def diff(f):
    def df(x, dx=0.01):
        return (f(x + dx) - f(x)) / dx
    return df

@diff
def d_square(x):  # square will be replaced by its derivative
    return x * x

print(d_square(1))
print(d_square(1, dx=1e-10))  # a better estimate

2.0100000000000007
2.000000165480742


This "wrapper function" pattern is the most common way to use decorators. For example, Python's built-in [`functools.cache`](https://docs.python.org/3/library/functools.html#functools.cache) decorator memoizes a function by wrapping it in a helper that first consults a lookup table.

In the art-deco pattern, we use this machinery to do something a little different. We write a decorator that takes `f` as input, treats `f`'s AST as code written in the DSL, and then generates new Python code by compiling `f` under the DSL's semantics. This creates the illusion of `f` "running" under the DSL's semantics.

# Parsing

Consider the "env" DSL I described earlier, which is like Python except all-caps variable names are looked up in the OS environment. In this section we will start with a "dummy" definition of an env function `f` (standing in for an input to a decorator), and parse it to produce an AST. Even though `f` is expressed as valid Python syntax, we will think of it as being env code.

Here is the definition of `f`:

In [3]:
def f():  # env code
    return "You are: " + USER

Our first move is to recover and parse `f`'s source. Amazingly, Python provides built-in tools for this, making it as easy as two lines of code.

In [4]:
import inspect
src = inspect.getsource(f)
print(repr(src))

import ast
print(ast.parse(src))

'def f():  # env code\n    return "You are: " + USER\n'


So far, so good! In principle, this is all we need to get started.

But to really do things _right_, we need to do a little bit more work. Importantly, we should save some useful metadata, such as source location, in order to give good error messages down the line.

In [5]:
src_file = inspect.getsourcefile(f)
lines, lineno = inspect.getsourcelines(f)

import os.path
print("f is defined in:", os.path.split(src_file)[-1])
print("f begins on line:", lineno)

f is defined in: 206327004.py
f begins on line: 1


There is also an important sublety to worry about. Python lets you define functions inside other syntactic constructs, and so definitions may be indented.

In [6]:
if True:
    def g():
        return "You are: " + USER
print(repr(inspect.getsource(g)))  # notice leading whitespace

'    def g():\n        return "You are: " + USER\n'


Such definitions are a problem because the parser will choke on the leading whitespace — that's not legal Python!

In [7]:
try:
    ast.parse(inspect.getsource(g))
except Exception as e:
    print(f'{e.__class__.__name__}: {e}')

IndentationError: unexpected indent (<unknown>, line 1)


To handle the general case, we have to first _dedent_ the source:

In [8]:
import textwrap
src_ = textwrap.dedent(src)

With all of that out of the way, we really _are_ ready to parse the source. Passing the `filename=` argument sets some of the source location information.

In [9]:
tree = ast.parse(src_, filename=src_file)
print(ast.dump(tree, indent=2))

Module(
  body=[
    FunctionDef(
      name='f',
      args=arguments(
        posonlyargs=[],
        args=[],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Return(
          value=BinOp(
            left=Constant(value='You are: '),
            op=Add(),
            right=Name(id='USER', ctx=Load())))],
      decorator_list=[],
      type_params=[])],
  type_ignores=[])


What we get back is a top-level `Module` node containing a single `FunctionDef` node. Typically, what we want to do is extract the `FunctionDef` inside that `Module` and work with that directly.

Another catch is that because we were working with the dummy function `f` here, the `decorator_list` is empty. But in general, when we use decorators, the current decorator will appear in `decorator_list`, and we typically want to remove it.

So let's perform both of those operations now…

In [10]:
tree = tree.body[0]  # module -> functiondef
if len(tree.decorator_list) > 0:
    tree.decorator_list.pop(0)

The final step is to finally fix-up source location information in the AST, in two steps:
1. The call to `ast.increment_lineno` correctly sets the line numbers in the source information (so that instead of being naively marked as "line 1", the first line of `f` is the actual `lineno` at which it appeared in the file).
2. The call to `ast_increment_colno` fixes column numbers, which got desynchronized from the true source locations when we dedented. Unfortunately, there is no analogous built-in `increment_colno` (to account for the dedenting we did earlier), but that is easy enough to write by hand.

In [11]:
ast.increment_lineno(tree, n=lineno - 1)

def ast_increment_colno(tree: ast.AST, n: int) -> None:
    for node in ast.walk(tree):
        if isinstance(node, ast.expr) or isinstance(node, ast.stmt):
            node.col_offset += n
            if node.end_col_offset is not None:
                node.end_col_offset += n

import re  # figure out how much we dedented by
lead_raw = re.match("^(.*)", src)
lead_fin = re.match("^(.*)", src_)
n_dedent = len(lead_raw.group()) - len(lead_fin.group())
ast_increment_colno(tree, n=n_dedent)

And that's it! We can print our final AST with source location information by passing `include_attributes=True` to `ast.dump`.

In [12]:
print(ast.dump(tree, include_attributes=True, indent=2))

FunctionDef(
  name='f',
  args=arguments(
    posonlyargs=[],
    args=[],
    kwonlyargs=[],
    kw_defaults=[],
    defaults=[]),
  body=[
    Return(
      value=BinOp(
        left=Constant(
          value='You are: ',
          lineno=2,
          col_offset=11,
          end_lineno=2,
          end_col_offset=22),
        op=Add(),
        right=Name(
          id='USER',
          ctx=Load(),
          lineno=2,
          col_offset=25,
          end_lineno=2,
          end_col_offset=29),
        lineno=2,
        col_offset=11,
        end_lineno=2,
        end_col_offset=29),
      lineno=2,
      col_offset=4,
      end_lineno=2,
      end_col_offset=29)],
  decorator_list=[],
  type_params=[],
  lineno=1,
  col_offset=0,
  end_lineno=2,
  end_col_offset=29)


To actually _use_ this source information, I recommend using the built-in `linecache` module. Here is starter code for a function that "cites" an AST node by printing the line it appears on and placing a caret at the start of that node.

In [13]:
def cite_node(node):
    import linecache
    line = linecache.getline(src_file, node.lineno)
    print(line[:-1])  # strip trailing \n
    print(' ' * node.col_offset + '^')

# cite the return statement
ret_stmt = tree.body[0]
cite_node(ret_stmt)

# cite the value being returned
ret_val = ret_stmt.value
cite_node(ret_val)

    return "You are: " + USER
    ^
    return "You are: " + USER
           ^


## Compiling

Once you have an AST, there are many ways you can start playing with it. One lightweight option, good for small source-to-source transformations, is to use the built-in [`ast.NodeTransformer`](https://docs.python.org/3/library/ast.html#ast.NodeTransformer) class to work with the AST directly. For the env DSL, we can easily write a `NodeTransformer` that transforms the env code to Python code by replacing references to all-caps variables (e.g. `USER`) with lookups in the OS environment.

In [14]:
class Envify(ast.NodeTransformer):
    def visit_Name(self, node):
        if not node.id.isupper(): return node
        return ast.Subscript(  # generates "os.environ[NAME]"
            value=ast.Attribute(ast.Name(id='os', ctx=ast.Load()), 'environ', ast.Load()),
            slice=ast.Constant(value=node.id),
            ctx=node.ctx
        )
Envify().visit(tree)
ast.fix_missing_locations(tree)
src_out = ast.unparse(tree)
print(src_out)

def f():
    return 'You are: ' + os.environ['USER']


The heavier-weight option is to translate the Python AST to a special AST for your DSL, and then to write a bespoke compiler for your DSL.

Regardless of how you go about this, you will ultimately have a string that represents your generated Python code. Once you have such a string, the last step is to compile it and return a handle to the compiled function to the caller.

Compiling is, in principle, as easy as calling `exec` on the string.

In [15]:
exec(src_out)
f()

'You are: kartikchandra'

But careful! This executes `src_out` in the scope of the _compiler_. You probably want to execute it in the scope of the _caller_, i.e. where the code was actually written. For example, you may want users of your DSL to be able to access that module's global variables. Calling `exec` naively would instead grant access to the _compiler's_ global variables.

To fix this, we need to access the caller's globals and pass them as arguments to `exec`.

In [16]:
exec(src_out, f.__globals__)
f()

'You are: kartikchandra'

But now we have another problem. Currently, we're able to call the updated `f()` because `src_out` defines `f` in the global namespace. But in general, we might not want decorators to write to (and pollute) the global namespace. Instead, we would like to run `src_out` in a special private namespace and then extract the defined `f`. This requires a third argument to `exec`:

In [17]:
retvals = {}
exec(src_out, f.__globals__, retvals)
f = retvals[f.__name__]
f()

'You are: kartikchandra'

Perfect! Let me summarize by packing all of the above into an actual decorator.

In [18]:
def env(f=None, **kwargs):
    # This trick lets us pass in optional arguments to the decorator via @env(...),
    # while still using @env without parentheses if no arguments are needed.
    if f is None: return lambda f: env(f, **kwargs)

    # get source
    import inspect
    src = inspect.getsource(f)
    src_file = inspect.getsourcefile(f)
    lines, lineno = inspect.getsourcelines(f)

    # fixup indentation
    import textwrap, re
    src_ = textwrap.dedent(src)
    lead_raw = re.match("^(.*)", src)
    lead_fin = re.match("^(.*)", src_)
    n_dedent = len(lead_raw.group()) - len(lead_fin.group())

    # parse
    import ast
    tree = ast.parse(src_, filename=src_file)

    # extract function body and remove current decorator
    tree = tree.body[0]
    if len(tree.decorator_list) > 0:
        tree.decorator_list.pop(0)

    # fixup source location metadata
    ast.increment_lineno(tree, n=lineno - 1)
    ast_increment_colno(tree, n=n_dedent)

    # actually implement the DSL!
    Envify().visit(tree)
    ast.fix_missing_locations(tree)
    src_out = 'import os\n' + ast.unparse(tree)  # "codegen"

    # process optional arguments, e.g. for debugging
    if kwargs.get('print_codegen', False):
        print('# Generated code:')
        print(src_out)
        print()

    # compile
    retvals = {}
    exec(src_out, f.__globals__, retvals)
    return retvals[f.__name__]

Using this DSL is as easy as applying the `@env` decorator to a function:

In [19]:
@env
def print_info():
    print(EDITOR, PAGER, SHELL)
print_info()

vim cat /bin/bash


We can also ask for the generated code by passing in the optional argument `print_codegen=True` to the decorator.

In [20]:
@env(print_codegen=True)
def is_good():
    print(EDITOR == 'vim')

# Generated code:
import os
def is_good():
    print(os.environ['EDITOR'] == 'vim')



### More on scope

Notice that because of our careful handling of scoping in `exec`, we can create definitions within another function's scope, and they don't "leak" into the global scope.

In [21]:
def make_printer():
    @env
    def printer():
        print(USER)
    return printer

try:
    make_printer()()
    printer()
except NameError as e:
    print(f'{e.__class__.__name__}: {e}')

kartikchandra
NameError: name 'printer' is not defined


This is good! It means we can write nice, modular code in Art-Deco DSLs. However, one issue is that the Art-Deco pattern doesn't let us access _local_ variables, because we only pass _globals_ into `exec`. For example, we would really like this to work, but it raises an error because `greeting` is local to `make_printer`:

In [22]:
def make_printer(greeting="Hello"):
    @env
    def printer():
        print(greeting, USER)
    return printer

try:
    make_printer()()
except NameError as e:
    print(f'{e.__class__.__name__}: {e}')

NameError: name 'greeting' is not defined


We might imagine capturing `greeting` by calling `locals()` in `env` and passing that into `exec` as well, but that is only a short-term fix. Even `locals()` cannot help us capture `greeting` in this case:

```python
def f(greeting="Hello"):
    def g():
        @env
        def printer():
            print(greeting, USER)
        return printer
    return g()
f()
```

because `greeting` is no longer part of `locals()` but rather inherited from the enclosing scope. Indeed, there is a good reason why variables like `greeting` should in general not be capturable at all from `exec`: we would like Python to be able to garbage-collect unused values after `f` executes, but if we allow referencing arbitrary bindings with `exec()` then Python would never be able to garbage-collect anything.

So… are we stuck? Do we have to disallow working in non-global scope? Ideally not: sometimes it really _is_ necessary to work in a non-global scope, such as when parametrically generating code in a library, or when writing multiple DSL functions that refer to each other in a mutually-recursive fashion.

In such cases, one strategy is to use what I call "virtual modules": dynamically-generated bespoke global namespaces where we can "install" any names we'd like.

Setting up a virtual module is a bit of a hassle, but here is some arcane starter code:

In [23]:
import importlib.abc, importlib.util

class StringLoader(importlib.abc.SourceLoader):
    def __init__(self, data):
        self.data = data
    def get_source(self, fullname):
        return self.data
    def get_data(self, path):
        return self.data.encode("utf-8")
    def get_filename(self, fullname):
        return "<dynamic>"

def make_module(name):
    loader = StringLoader('''def install(x): exec(x, globals()); return globals()''')
    spec = importlib.util.spec_from_loader(name, loader, origin="built-in")
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

Now, we can initialize and write to a virtual module.

In [24]:
m = make_module('my_virtual_module')
print(m)

m.greeting = "Hello"
dir(m)

<module 'my_virtual_module' (built-in)>


['__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'greeting',
 'install']

We can "install" a definition in the module's namespace by calling `install`.

In [25]:
f = m.install('''
def f(): print(greeting, "world!")
''')['f']

f()

Hello world!


Notice that `f` now _can_ capture `greeting` from the module `m`'s global scope. Similarly, `f` would be able to refer to a function `g` that we are yet to define, as long a `g` is installed into the same module `m`.

The rest is plumbing: we just need to forward relevant variables to `m`, and then run `m.install` instead of `exec` inside our decorator.

# Conclusion

In this guide I summarized the key ideas behind building Art-Deco DSLs. I showed you how to work with decorators, how to parse input Python code (being careful about source location), and how to run generated Python code (being careful about scope). I hope you find these tools useful as you go about building your own Art-Deco DSLs!